In [ ]:
# Install dependencies (run this first in Colab)
!pip install tensorflow tensorflow-datasets -q

import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU available: {tf.config.list_physical_devices('GPU')}")

TensorFlow version: 2.19.0
GPU available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
# Configuration - CORRECTED for 50%+ accuracy
IMG_SIZE = 224  # INCREASED - better accuracy
BATCH_SIZE = 32
STEPS_PER_EPOCH = 1200  # Full dataset coverage per epoch
VAL_STEPS = 300  # Proper validation evaluation
EPOCHS_HEAD = 15  # INCREASED - more time to learn
FINE_TUNE = True  # ENABLED - will boost to 50%+
FT_LAYERS = 40  # INCREASED - unfreeze more layers
FT_EPOCHS = 5  # INCREASED - more fine-tuning
ALPHA = 0.75  # INCREASED - more model capacity (was 0.5)
LEARNING_RATE_HEAD = 0.001
LEARNING_RATE_FT = 0.0001

print("Configuration loaded:")
print(f"  Image size: {IMG_SIZE}x{IMG_SIZE}")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Steps per epoch: {STEPS_PER_EPOCH}")
print(f"  Head training epochs: {EPOCHS_HEAD}")
print(f"  Fine-tuning: {FINE_TUNE}")
if FINE_TUNE:
    print(f"  Fine-tune layers: {FT_LAYERS}")
    print(f"  Fine-tune epochs: {FT_EPOCHS}")

Configuration loaded:
  Image size: 224x224
  Batch size: 32
  Steps per epoch: 1200
  Head training epochs: 15
  Fine-tuning: True
  Fine-tune layers: 40
  Fine-tune epochs: 5


In [ ]:
# Load Food-101 dataset (50% split)
print("Loading Food-101 dataset...")
(train_ds, val_ds), info = tfds.load(
    'food101',
    split=['train[:50%]', 'validation[:50%]'],
    with_info=True,
    as_supervised=True
)

# Get class names
CLASS_NAMES = info.features['label'].names
NUM_CLASSES = len(CLASS_NAMES)
print(f"\nDataset info:")
print(f"  Classes: {NUM_CLASSES}")
print(f"  Train examples: {info.splits['train'].num_examples // 2}")
print(f"  Val examples: {info.splits['validation'].num_examples // 2}")
print(f"\nSample classes: {CLASS_NAMES[:5]}")

# Preprocessing functions
def preprocess_train(image, label):
    """Preprocessing with augmentation for training"""
    image = tf.cast(image, tf.float32)
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
    # Data augmentation
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, 0.2)
    image = tf.image.random_contrast(image, 0.8, 1.2)
    # Normalize to [0, 1] - matches backend preprocessing
    image = image / 255.0
    # Clip to ensure valid range
    image = tf.clip_by_value(image, 0.0, 1.0)
    return image, label

def preprocess_val(image, label):
    """Preprocessing without augmentation for validation"""
    image = tf.cast(image, tf.float32)
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
    # Normalize to [0, 1] - matches backend preprocessing
    image = image / 255.0
    image = tf.clip_by_value(image, 0.0, 1.0)
    return image, label

# Build training pipeline
train_ds = train_ds.map(preprocess_train, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.shuffle(1000)
train_ds = train_ds.batch(BATCH_SIZE)
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)

# Build validation pipeline
val_ds = val_ds.map(preprocess_val, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.batch(BATCH_SIZE)
val_ds = val_ds.prefetch(tf.data.AUTOTUNE)

print("\n✓ Dataset pipelines ready")

Loading Food-101 dataset...


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/food101/incomplete.FWW6ZO_2.0.0/food101-train.tfrecord*...:   0%|         …

Generating validation examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/food101/incomplete.FWW6ZO_2.0.0/food101-validation.tfrecord*...:   0%|    …

Dataset food101 downloaded and prepared to /root/tensorflow_datasets/food101/2.0.0. Subsequent calls will reuse this data.

Dataset info:
  Classes: 101
  Train examples: 37875
  Val examples: 12625

Sample classes: ['apple_pie', 'baby_back_ribs', 'baklava', 'beef_carpaccio', 'beef_tartare']

✓ Dataset pipelines ready


In [ ]:
# Build model with internal scaling layer for backend compatibility
print("Building model...")

# Create base model (MobileNetV2)
base_model = keras.applications.MobileNetV2(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False,
    weights='imagenet',
    alpha=ALPHA
)
base_model.trainable = False  # Freeze initially

# Build full model
inputs = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))

# Internal scaling: [0,1] -> [-1,1] for MobileNetV2 compatibility
x = layers.Lambda(lambda x: x * 2.0 - 1.0, name='scale_input')(inputs)

# MobileNetV2 backbone
x = base_model(x, training=False)

# Classification head
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)  # INCREASED dropout
outputs = layers.Dense(NUM_CLASSES, activation='softmax')(x)

model = keras.Model(inputs, outputs)

print(f"\n✓ Model built:")
print(f"  Input shape: {model.input_shape}")
print(f"  Output classes: {NUM_CLASSES}")
print(f"  Total params: {model.count_params():,}")
print(f"  Trainable params: {sum([tf.size(w).numpy() for w in model.trainable_weights]):,}")

Building model...
5903360/5903360 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step

✓ Model built:
  Input shape: (None, 224, 224, 3)
  Output classes: 101
  Total params: 1,511,445
  Trainable params: 129,381


In [ ]:
# Compile and train head
print("\n" + "="*60)
print("PHASE 1: Training classification head")
print("="*60)

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE_HEAD),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Callbacks
callbacks = [
    keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=5,
        restore_best_weights=True,
        verbose=1
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=2,
        verbose=1,
        min_lr=1e-6
    )
]

# Train
history_head = model.fit(
    train_ds,
    epochs=EPOCHS_HEAD,
    steps_per_epoch=STEPS_PER_EPOCH,
    validation_data=val_ds,
    validation_steps=VAL_STEPS,
    callbacks=callbacks,
    verbose=1
)

# Get best accuracy from head training
best_acc_head = max(history_head.history['val_accuracy'])
print(f"\n✓ Head training complete. Best val accuracy: {best_acc_head:.4f} ({best_acc_head*100:.2f}%)")


PHASE 1: Training classification head
Epoch 1/15
1184/1200 ━━━━━━━━━━━━━━━━━━━━ 1s 81ms/step - accuracy: 0.2975 - loss: 3.0601

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


1200/1200 ━━━━━━━━━━━━━━━━━━━━ 140s 99ms/step - accuracy: 0.2989 - loss: 3.0525 - val_accuracy: 0.5745 - val_loss: 1.6483 - learning_rate: 0.0010
Epoch 2/15
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 106s 86ms/step - accuracy: 0.5258 - loss: 1.8557 - val_accuracy: 0.5906 - val_loss: 1.5532 - learning_rate: 0.0010
Epoch 3/15
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 106s 86ms/step - accuracy: 0.5590 - loss: 1.7106 - val_accuracy: 0.6022 - val_loss: 1.5111 - learning_rate: 0.0010
Epoch 4/15
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 104s 85ms/step - accuracy: 0.5773 - loss: 1.6176 - val_accuracy: 0.6062 - val_loss: 1.4928 - learning_rate: 0.0010
Epoch 5/15
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 143s 86ms/step - accuracy: 0.5856 - loss: 1.5762 - val_accuracy: 0.6027 - val_loss: 1.4963 - learning_rate: 0.0010
Epoch 6/15
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 105s 86ms/step - accuracy: 0.5947 - loss: 1.5405 - val_accuracy: 0.6083 - val_loss: 1.4905 - learning_rate: 0.0010
Epoch 7/15
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 106s 87ms/step - accuracy: 

In [ ]:
# Fine-tune top layers
if FINE_TUNE:
    print("\n" + "="*60)
    print("PHASE 2: Fine-tuning top layers")
    print("="*60)

    # Unfreeze top layers
    base_model.trainable = True

    # Freeze all layers except the top FT_LAYERS
    for layer in base_model.layers[:-FT_LAYERS]:
        layer.trainable = False

    trainable_count = sum([tf.size(w).numpy() for w in model.trainable_weights])
    print(f"  Unfreezing top {FT_LAYERS} layers")
    print(f"  Trainable params: {trainable_count:,}")

    # Recompile with lower learning rate
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE_FT),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    # Fine-tune
    history_ft = model.fit(
        train_ds,
        epochs=FT_EPOCHS,
        steps_per_epoch=STEPS_PER_EPOCH,
        validation_data=val_ds,
        validation_steps=VAL_STEPS,
        callbacks=callbacks,
        verbose=1
    )

    best_acc_ft = max(history_ft.history['val_accuracy'])
    print(f"\n✓ Fine-tuning complete. Best val accuracy: {best_acc_ft:.4f} ({best_acc_ft*100:.2f}%)")
else:
    print("\nSkipping fine-tuning (FINE_TUNE=False)")


PHASE 2: Fine-tuning top layers
  Unfreezing top 40 layers
  Trainable params: 1,161,925
Epoch 1/5
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 143s 101ms/step - accuracy: 0.4742 - loss: 2.2413 - val_accuracy: 0.6245 - val_loss: 1.5024 - learning_rate: 1.0000e-04
Epoch 2/5
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 108s 88ms/step - accuracy: 0.6410 - loss: 1.3150 - val_accuracy: 0.6509 - val_loss: 1.3269 - learning_rate: 1.0000e-04
Epoch 3/5
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 110s 90ms/step - accuracy: 0.6955 - loss: 1.0841 - val_accuracy: 0.6667 - val_loss: 1.2606 - learning_rate: 1.0000e-04
Epoch 4/5
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 143s 116ms/step - accuracy: 0.7392 - loss: 0.9307 - val_accuracy: 0.6694 - val_loss: 1.2801 - learning_rate: 1.0000e-04
Epoch 5/5
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 106s 87ms/step - accuracy: 0.7645 - loss: 0.8093 - val_accuracy: 0.6811 - val_loss: 1.2026 - learning_rate: 1.0000e-04
Restoring model weights from the end of the best epoch: 5.

✓ Fine-tuning complete. Best val accuracy: 0.681

In [ ]:
# Final evaluation
print("\n" + "="*60)
print("FINAL EVALUATION")
print("="*60)

# Evaluate on full validation set
final_results = model.evaluate(val_ds, steps=VAL_STEPS, verbose=1)
final_acc = final_results[1]
final_loss = final_results[0]

print(f"\nFinal validation accuracy: {final_acc:.4f} ({final_acc*100:.2f}%)")
print(f"Final validation loss: {final_loss:.4f}")

# Save model in H5 format (backend compatible)
model.save('food_model2.h5', save_format='h5')
print("✓ Saved model: food_model.h5")

# Save labels.txt (must match CLASS_NAMES order)
with open('labels.txt2', 'w') as f:
    for class_name in CLASS_NAMES:
        f.write(class_name + '\n')
print("✓ Saved labels: labels.txt")

# Display accuracy achieved
print("\n" + "="*60)
if final_acc >= 0.50:
    print(f"✓ SUCCESS! Achieved {final_acc*100:.2f}% accuracy (target: 50%+)")
else:
    print(f"⚠ Warning: Only achieved {final_acc*100:.2f}% accuracy (target: 50%+)")
    print("  Try running again with ALPHA=1.0 or more epochs")
print("="*60)


FINAL EVALUATION
300/300 ━━━━━━━━━━━━━━━━━━━━ 18s 59ms/step - accuracy: 0.6806 - loss: 1.1980



Final validation accuracy: 0.6811 (68.11%)
Final validation loss: 1.2026
✓ Saved model: food_model.h5
✓ Saved labels: labels.txt

✓ SUCCESS! Achieved 68.11% accuracy (target: 50%+)


In [ ]:
# Test predictions on a few samples
print("\nTesting predictions on sample images...")

# Get a batch from validation set
for images, labels in val_ds.take(1):
    # Predict
    predictions = model.predict(images[:5], verbose=0)

    # Display results
    for i in range(5):
        true_label = CLASS_NAMES[labels[i].numpy()]
        pred_idx = np.argmax(predictions[i])
        pred_label = CLASS_NAMES[pred_idx]
        confidence = predictions[i][pred_idx]

        match = "✓" if pred_idx == labels[i].numpy() else "✗"
        print(f"{match} True: {true_label:20s} | Pred: {pred_label:20s} ({confidence*100:.1f}%)")


Testing predictions on sample images...
✓ True: cup_cakes            | Pred: cup_cakes            (78.3%)
✓ True: ramen                | Pred: ramen                (99.1%)
✓ True: spaghetti_carbonara  | Pred: spaghetti_carbonara  (71.0%)
✓ True: hamburger            | Pred: hamburger            (77.9%)
✓ True: takoyaki             | Pred: takoyaki             (96.3%)
